In [0]:
#mapeia pastas
bronze_path = "/Volumes/bikestore/default/1_bronze"
silver_path = "/Volumes/bikestore/default/2_silver"
gold_path = "/Volumes/bikestore/default/3_gold"
origem_path = "/Volumes/bikestore/default/4_origem"
destino_path = "/Volumes/bikestore/default/5_destino"

In [0]:
# cria tabelas bronzes temporarias
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))

In [0]:
df = spark.sql("""
select
a.order_id,
a.customer_id,
a.order_status,
a.order_date,
a.required_date,
a.shipped_date,
a.store_id,
a.staff_id,
c.first_name,
c.email,
c.active,
b.store_name,
b.city,
b.state
from tmp_bronze_orders a
left join tmp_bronze_stores b on a.store_id = b.store_id
left join tmp_bronze_staffs c on a.staff_id = c.staff_id
""")

display(df)

In [0]:
# salva o dataframe como tabela
df.write\
    .mode("overwrite")\
    .format("delta")\
    .option("mergeSchema", "true")\
    .saveAsTable("bikestore.default.s_orders")

del(df)

In [0]:
%sql
select *
from bikestore.default.s_orders
limit 3